<a href="https://colab.research.google.com/github/javier-jaime/Tool-Crib/blob/master/LangChain/HuggingFaceHub_LLMs_integrations_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain & HuggingFaceHub LLMs integrations Tutorial

In this tutorial we will use LangChain Open Source Orchestration Framework to integrate Large Language Models from Hugging Face Hub.

This tutorial is based on code from: Generative AI with LangChain: Build large language model (LLM) apps with Python, ChatGPT and other LLMs by Ben Auffarth (2023) https://github.com/benman1/generative_ai_with_langchain

### Notebook Preparation

First we will need to install the langchain library, and import all the necessary requirements

In [21]:
# Install langchain library the first time, comment/uncomment as required
!pip install langchain
from langchain_community.llms import HuggingFaceHub
from langchain import PromptTemplate, LLMChain

# Import operating system and userdata interface from Google Colab
import os
from google.colab import userdata

To use Hugging Face as a provider for your models, you need to create an account and get an API Key from: https://huggingface.co/settings/profile

In [3]:
# Set an API key directly to the Python environment, comment/uncomment as required

# os.environ['HUGGINGFACEHUB_API_TOKEN'] = '<your API key token>'

# Or store the environment variable as a Secret in Colab, and

# Access your Secret Key and declare it as an environment variable

os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HUGGINGFACEHUB_API_TOKEN')

### Hugging Face Simple Example

In this case we are using a simple open source model developed by Google to test a simple prompt,

with a temperature of 0.5 (half BS) and a maximum lenght of 64.

In [56]:
# Initialize the model
llm = HuggingFaceHub(
model_kwargs={"temperature": 0.5, "max_length": 64},
repo_id="google/flan-t5-xxl"
)

In [52]:
# Pass the prompt
prompt = input('Enter your question: ')
completion = llm(prompt)
print(completion)

Enter your question: Where is Calgary?
Canada


Now we can run a simple model by chaining a prompt template and a LLM

In [57]:
template = """Question: {question}
Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=['question'])

llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)

test_question = " Who was the mayor in the year of Calgary Olympics games"

llm_chain.run(test_question)



> Entering new LLMChain chain...
Prompt after formatting:
Question:  Who was the mayor in the year of Calgary Olympics games
Answer: Let's think step by step.

> Finished chain.


'In 1988, the Calgary Olympics games were held. The mayor in 1988 was Ken Melamed. The answer: Ken Melamed.'

We used verbose=True to see the reasoning, but we didn't ge the right result, Ken Melamed was the Major of Vancouver in 1988. Let's try again.

In [50]:
question = " Who was the premier of Alberta in the year of Calgary Olympics games"
llm_chain.run(question)



> Entering new LLMChain chain...
Prompt after formatting:
Question:  Who was the premier of Alberta in the year of Calgary Olympics games
Answer: Let's think step by step.

> Finished chain.


'In the year of the Calgary Olympics games, Ralph Klein was the premier of Alberta. Ralph Klein was the premier of Alberta from 1989-1993. The answer: Ralph Klein.'

Ralph was the Mayor not the Premier during the Calgary Olympics, close enough.
